In [4]:
import numpy as np
import theano
import theano.tensor as T
from triple_reader import triple_reader
from question_reader import question_reader

In [6]:
# read in file as tensors
text_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/"
    "Senior Thesis Code/ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/1/Triples/1-medium.txt")
    
tr = triple_reader(text_file)
print tr.tripleList
tensor = tr.tensor
enMap = tr.enMap
relMap = tr.relMap

# start to build bag-of-words

[['critic' 'recalls' 'painting']
 ['novelist' 'recalls' 'painting']
 ['I' 'was' 'bought']
 ['painting' 'was' 'Fell']
 ['anyone' 'read' 'books']
 ['anyone' 'be' 'read']
 ['I' 'went' 'books']
 ['I' 'be' 'went']
 ['writers' 'need' 'books']
 ['writers' 'be' 'need']
 ['she' 'feeling' 'same']
 ['I' 'imagined' 'same']
 ['I' 'was' 'wrong']
 ['she' 'sold' 'them']
 ['she' 'sold' 'what']
 ['Sheila' 'care' 'them']
 ['Sheila' 'care' 'what']
 ['they' 'thought' 'them']
 ['they' 'thought' 'what']
 ['who' 'looked' 'them']
 ['who' 'looked' 'what']
 ['She' 'ease' 'work']
 ['She' 'was' 'ease']
 ['that' 'made' 'canvas']
 ['that' 'made' 'me']
 ['confidence' 'was' 'canvas']
 ['confidence' 'was' 'me']
 ['she' 'propped' 'canvas']
 ['she' 'propped' 'me']
 ['canvas' 'relax' 'canvas']
 ['canvas' 'relax' 'me']
 ['need' 'was' 'Cumberland']
 ['need' 'been' 'was']
 ['I' 'apprehensive' 'Cumberland']
 ['I' 'been' 'apprehensive']
 ['I' 'know' 'Cumberland']
 ['I' 'been' 'know']
 ['we' 'had' 'Cumberland']
 ['we' 'been' 'h

In [10]:
q_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/Senior Thesis Code/"
"ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/1/1-medium1-q.txt")

qr = question_reader(q_file)

questions = qr.questionCombos
for question in questions:
    # For each question, convert it to an entity-?-entity query or entity-relation-? query
    q_words = question[0].split(" ")
#     print question[0]
#     print q_text
    choices = question[1:]
    for word in q_words:
        print word
    for choice in choices:
        choice_words = choice.split(" ")

 I n 
 l i n e 
 2 5 , 
 t h e 
 a u t h o r 
 a s s u m e s 
 t h a t 
 " j u s t i c e " 
 w o u l d 
 b e 
 
 
 
 
 r e c o g n i z i n g 
 T h e 
 u n i q u e 
 a c h i e v e m e n t s 
 o f 
 a n 
 a r t i s t 
 e n s u r i n g 
 t h a t 
 a 
 w o r k 
 o f 
 a r t 
 r e a c h e s 
 t h e 
 w i d e s t 
 p o s s i b l e 
 a u d i e n c e 
 d i s p l a y i n g 
 a 
 w o r k 
 o f 
 a r t 
 t o 
 i t s 
 b e s t 
 a d v a n t a g e 
 e n h a n c i n g 
 o n e ' s 
 d a i l y 
 l i f e 
 w i t h 
 b e a u t i f u l 
 a r t 
 p r o v i d i n g 
 e l e g a n t 
 s u r r o u n d i n g s 
 f o r 
 e x c e p t i o n a l 
 p a i n t i n g s 
 " I t 
 w a s 
 a 
 t e r r i b l e 
 m i s t a k e " 
 ( l i n e 
 3 6 ) 
 b e c a u s e 
 t h e 
 n a r r a t o r 
 h a d 
 n o 
 o t h e r 
 s o u v e n i r s 
 o f 
 C u m b e r l a n d 
 a l l o w e d 
 p r a g m a t i c 
 c o n c e r n s 
 t o 
 o v e r r i d e 
 h e r 
 f o n d n e s s 
 f o r 
 t h e 
 p a i n t i n g 
 d i d 
 n o t 
 r e a l

In [7]:
# Create a weight matrix of given size. 
# The matrix is initialized randomly with Gaussian distribution 
# with mean=0 and \sigma=0.1
def initializeWeightMatrix(in_size, out_size):
    return theano.shared(np.random.randn(in_size, out_size))

# Create a bias vector of all zeros of given size
def initializeBiasVector(size):
    return theano.shared(np.zeros(size,))

In [ ]:
# Initialize all our parameters, given our dimensions.
# A is the first matrix used to embed our input. It has size Vxd
# B is the matrix used to embed the query. It has size Vxd
# C is the next matrix used to embed our input. It has size Vxd
# W is the final matrix. Takes output O and produces result R. It has size dxV
# H is the linear mapping we apply to u_i in each layer, since we have set it
#      up like an RNN
def initializeParams(d, V):
    A = initializeWeightMatrix(V, d)
    B = initializeWeightMatrix(V, d)
    C = initializeWeightMatrix(V, d)
    W = initializeWeightMatrix(d, V)
    H = initializeWeightMatrix(d, d) #H is just a vector
    
#     A = theano.shared(initializeWeightMatrix(d, V))
#     B = theano.shared(initializeWeightMatrix(d, V))
#     C = theano.shared(initializeWeightMatrix(d, V))
#     W = theano.shared(initializeWeightMatrix(V, d))
    return A, B, C, W, H

A, B, C, W, H = initializeParams(d, V)
weightMatrices = [A, B, C, W, H]
# for weightMatrix in weightMatrices:
#     print weightMatrix.eval()
# print(theano.shared(np.zeros((1,V), dtype=np.int64)).dot(B).shape.eval())